In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from functions import *
from sklearn.model_selection import train_test_split
import warnings

In [12]:
socio = pd.read_feather('BF_Dados_Socio_Economicos.feather')
socio = socio.drop_duplicates('identificação')
cont = pd.read_feather('BF_Dados_Contratos.feather')
cont = cont.loc[cont['CONTRATO'] > 40000]
df = cont.join(socio.set_index('identificação'),on = 'IDENTIFICAÇÃO')
df = df.loc[~df['atividade'].isna()]
df['DEFAULT'] = df['ATRASO_DIAS'] > 90
df.loc[df['numero_de_pessoas_na_casa'] > 10,'numero_de_pessoas_na_casa'] = 10
serie = df['numero_de_pessoas_na_casa'].dropna()
df.loc[df['numero_de_pessoas_na_casa'].isna(),'numero_de_pessoas_na_casa'] = np.random.normal(serie.mean(),serie.std(),df['numero_de_pessoas_na_casa'].isna().sum())
df['numero_de_pessoas_na_casa'] = df['numero_de_pessoas_na_casa'].apply(lambda x: round(abs(x)))
df = df.dropna().sample(frac = 1).reset_index(drop = True)
to_drop = ['SITUACAO','QUANTIDADE_PAGAS','ATRASO_DIAS','ATRASO_MAXIMO','RENEGOCIADO','BAIRRO','CONTRATO','IDENTIFICAÇÃO','CEP','melhor_data_vencimento']
df = df.drop(to_drop,axis = 1)

### Transformação e separação treino-teste

In [15]:
df = numerize_dummie(df)
x_train,x_test,y_train,y_test = train_test_split(df.drop('DEFAULT',axis = 1).values,df['DEFAULT'].values,test_size = 0.25)

### Definição de métrica
f-beta score

In [71]:
from sklearn.metrics import fbeta_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

peso_recall = 2
metric = lambda real,pred: fbeta_score(real,pred,beta = peso_recall)

In [64]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print('Métricas para previsão de 100% negativo:')
    print('Score:','%.3f' % metric(y_test,np.zeros(y_test.shape)))
    print('Precision:','%.3f' % precision_score(y_test,np.zeros(y_test.shape)))
    print('Recall:','%.3f' % recall_score(y_test,np.zeros(y_test.shape)))
    print('\n')
    print('Métricas para previsão de 100% positivo:')
    print('Score:','%.3f' % metric(y_test,np.ones(y_test.shape)))
    print('Precision:','%.3f' % precision_score(y_test,np.ones(y_test.shape)))
    print('Recall:','%.3f' % recall_score(y_test,np.ones(y_test.shape)))

Métricas para previsão de 100% negativo:
Score: 0.000
Precision: 0.000
Recall: 0.000


Métricas para previsão de 100% positivo:
Score: 0.173
Precision: 0.040
Recall: 1.000


#### Modelo 0: Simple XGBoost

In [65]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier().fit(x_train, y_train)
prediction = xgb_model.predict(x_test)
print('Score:','%.3f' % metric(y_test,prediction))
print('Precision:','%.3f' % precision_score(y_test,prediction))
print('Recall:','%.3f' % recall_score(y_test,prediction))

Score: 0.023
Precision: 0.412
Recall: 0.019


#### Modelo 1: Random Forest

In [67]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(x_train,y_train)
pediction = rf_model.predict(x_test)
print('Score:','%.3f' % metric(y_test,prediction))
print('Precision:','%.3f' % precision_score(y_test,prediction))
print('Recall:','%.3f' % recall_score(y_test,prediction))

Score: 0.032
Precision: 0.278
Recall: 0.027


#### Modelo 2: XGBoost com métrica ajustada

In [75]:
results = {}
for value in [0,1,2,3,4,5,6,7,8,9,10]:
    xgb_model = XGBClassifier(scale_pos_weight = value).fit(x_train, y_train)
    prediction = xgb_model.predict(x_test)
    results[metric(y_test,prediction)] = value
best_score = max(list(results.keys()))
print('Melhor Parâmetro:',results[best_score])
xgb_model = XGBClassifier(scale_pos_weight = results[best_score]).fit(x_train, y_train)
prediction = xgb_model.predict(x_test)
print('Score:','%.3f' % metric(y_test,prediction))
print('Precision:','%.3f' % precision_score(y_test,prediction))
print('Recall:','%.3f' % recall_score(y_test,prediction))

Melhor Parâmetro: 9
Score: 0.277
Precision: 0.185
Recall: 0.317


#### Modelo 3: SMOTE oversampling

In [66]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
x_smote, y_smote = smote.fit_resample(x_train, y_train)
xgb_model = XGBClassifier().fit(x_smote, y_smote)
prediction = xgb_model.predict(x_test)
print('Score:','%.3f' % metric(y_test,prediction))
print('Precision:','%.3f' % precision_score(y_test,prediction))
print('Recall:','%.3f' % recall_score(y_test,prediction))

Score: 0.032
Precision: 0.278
Recall: 0.027


#### Modelo 4: Tomek Links Undersampling

In [68]:
from imblearn.under_sampling import TomekLinks

tl = TomekLinks(sampling_strategy='majority')
x_tl, y_tl = tl.fit_resample(x_train, y_train)
xgb_model = XGBClassifier().fit(x_tl, y_tl)
prediction = xgb_model.predict(x_test)
print('Score:','%.3f' % metric(y_test,prediction))
print('Precision:','%.3f' % precision_score(y_test,prediction))
print('Recall:','%.3f' % recall_score(y_test,prediction))

Score: 0.029
Precision: 0.360
Recall: 0.024


#### Modelo 5: XGBoost ajustado + SMOTE

In [77]:
smote = SMOTE()
x_smote, y_smote = smote.fit_resample(x_train, y_train)
xgb_model = XGBClassifier(scale_pos_weight = 9).fit(x_smote, y_smote)
prediction = xgb_model.predict(x_test)
print('Score:','%.3f' % metric(y_test,prediction))
print('Precision:','%.3f' % precision_score(y_test,prediction))
print('Recall:','%.3f' % recall_score(y_test,prediction))

Score: 0.293
Precision: 0.159
Recall: 0.371


In [85]:
print('Percentual de Falsos Positivos:')
print(100 * ((prediction != y_test) & (y_test == 0)).sum() / (y_test == 0).sum())

Percentual de Falsos Positivos:
8.170731707317072


#### Conclusão:
O melhor modelo foi XGBoost com scale_pos_weight = 9 e rebalanceamento de classes com modelo SMOTE

##### Características do modelo:
37% dos futuros inadimplentes são identificados, porém 8% dos futuros adimplentes são erroneamente excluídos